### Manage Imports

In [ ]:
import keras
%matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import re
import numpy as np
from sklearn import svm, metrics
from keras.preprocessing.image import img_to_array, load_img
from keras.applications import VGG16
from keras import layers, models, optimizers
keras.__version__

### Globals

In [ ]:
users = ["alex", "ben", "miao", "natasha", "nick", "sarah", "sean", "spencer", "tim", "yijun"]
mats = ["cloth", "concrete", "door", "drywall", "laminant", "whiteboard"]
pressures = ["hard", "soft"]
#hello

AUG_FACTOR = 8
IMAGE_SIZE = 128
FRAMES_PER_VID = 8
EPOCHS = 50

#### Optionally Remove Hand:

In [ ]:
remove_hand = True
if remove_hand:
    frames = "swipe_frames"
else:
    frames = "frames"

# Single Frame Classification

### Load Data

In [ ]:
# perform leave-one-out classification for each user in turn
#for user in users:
for user in ["alex"]:
    print "User:", user
    
    # training data consists of swipes from all other users (for mats of interest)
    print "\tFinding training images..."
    files = []
    training_users = [u for u in users if u != user]
    for train_user in training_users:
        for mat in mats:
            for pres in pressures:
                folder = "data/"+train_user+"/"+frames+"/"+mat+"/"+pres+"/"
                files.extend([folder+f for f in os.listdir(folder)])
    files.sort() # only keeps video frames in order if < 10 frames per video
    print "\tTraining images found:", len(files)
    
    # create single arrays to store data (use 224x224 for VGGNet)
    X_train = np.ndarray(shape=(len(files)*AUG_FACTOR, IMAGE_SIZE, IMAGE_SIZE), dtype=np.float32)
    y_train = np.ndarray(shape=(len(files)*AUG_FACTOR, 1), dtype=np.float32)

    # load in training data from images into normalized array
    print "\tLoading and augmenting training images..."
    n = 0
    for f in files:
        # read in, normalize, resize for VGGNet
        x = cv2.resize(cv2.imread(f, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        X_train[n] = x / 255.0

        # perform data augmentation
        X_train[n+1] = np.fliplr(X_train[n])
        X_train[n+2] = np.rot90(X_train[n])
        X_train[n+3] = np.fliplr(X_train[n+2])
        X_train[n+4] = np.rot90(X_train[n+2])
        X_train[n+5] = np.fliplr(X_train[n+4])
        X_train[n+6] = np.rot90(X_train[n+4])
        X_train[n+7] = np.fliplr(X_train[n+6])

        # assign labels
        if "/hard/" in f:
            y_train[n:n+AUG_FACTOR] = np.ones((AUG_FACTOR,1))
        else:
            y_train[n:n+AUG_FACTOR] = np.zeros((AUG_FACTOR,1))
        n += AUG_FACTOR
    print "\tAll", n, "training images and labels stored successfully!"
    
    # find all test data
    print "\tFinding test images..."
    test_files = []
    for mat in mats:
        for pres in pressures:
            folder = "data/"+user+"/"+frames+"/"+mat+"/"+pres+"/"
            test_files.extend([folder+f for f in os.listdir(folder)])
    test_files.sort() # only keeps video frames in order if < 10 frames per video
    print "\tTest images found:", len(test_files)
    
    # load in test data
    X_test = np.ndarray(shape=(len(test_files), IMAGE_SIZE, IMAGE_SIZE), dtype=np.float32)
    y_test = np.ndarray(shape=(len(test_files), 1), dtype=np.float32)
    print "\tLoading test images..."
    n2 = 0
    for f in test_files:
        x = cv2.resize(cv2.imread(f, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        X_test[n2] = x / 255.0
        y_test[n2] = float("/hard/" in f)
        n2 += 1
    print "\tAll", n2, "test images and labels stored successfully!"

In [ ]:
fig = plt.figure(figsize=(8,4)) # Notice the equal aspect ratio
ax = [fig.add_subplot(2,4,i+1) for i in range(8)]

for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(X_train[i], cmap='gray')

for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')

fig.subplots_adjust(wspace=0, hspace=0)

## Shallow Single-Frame CNN (from scratch)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(IMAGE_SIZE,IMAGE_SIZE,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2,2)))
#model.add(layers.Dropout(0.25))
#model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))

# fully connected
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

X_train = np.reshape(X_train, (X_train.shape[0],IMAGE_SIZE,IMAGE_SIZE,1))
history = model.fit(X_train, y_train,
          validation_split=0.2,
          batch_size=100,
          epochs=EPOCHS,
          verbose=1
          )

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],IMAGE_SIZE,IMAGE_SIZE,1))
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Single Frame Accuracy:')
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Single-Frame Shallow CNN Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

In [ ]:
# create feature arrays from earlier per-frame predictions
y_train_frame_pred = model.predict(X_train)
X_train_svm = np.reshape(np.rollaxis(np.reshape(
    y_train_frame_pred, (len(files)//FRAMES_PER_VID,FRAMES_PER_VID,AUG_FACTOR)),1,3), (len(files),FRAMES_PER_VID))
y_train_svm = y_train[::FRAMES_PER_VID]

# same for test set
y_test_frame_pred = model.predict(X_test)
X_test_svm = np.reshape(y_test_frame_pred, (len(test_files)//FRAMES_PER_VID, FRAMES_PER_VID))
y_test_svm = y_test[::FRAMES_PER_VID]

In [ ]:
# take average prediction, round at end
simple_y_pred = np.round(np.sum(X_test_svm, axis=1)/FRAMES_PER_VID)
avg_acc = metrics.accuracy_score(y_test_svm, simple_y_pred)

# train SVM to make prediction
clf = svm.SVC()
clf.fit(X_train_svm, y_train_svm)
svm_acc = clf.score(X_test_svm, y_test_svm)

print('Per-Video AVG Accuracy:', avg_acc)
print('Per-Video SVM accuracy:', svm_acc)

## Deep Single-Frame CNN (from scratch)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(IMAGE_SIZE,IMAGE_SIZE, 1)))
model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# fully connected
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

X_train = np.reshape(X_train, (X_train.shape[0],IMAGE_SIZE,IMAGE_SIZE,1))
history = model.fit(X_train, y_train,
          validation_split=0.2,
          batch_size=100,
          epochs=EPOCHS,
          verbose=1
          )

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],IMAGE_SIZE,IMAGE_SIZE,1))
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Single-Frame Deep CNN Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Single-Frame CNN pre-training on VGG

In [ ]:
vgg = VGG16(weights='imagenet',
           include_top=False,
           input_shape=(224,224,3))

# feed each image through VGG and obtain 7*7*512 features
X_train_feat = np.zeros(shape=(X_train.shape[0],7,7,512))
print 'Total training images:', X_train.shape[0]
for i in range(X_train.shape[0]):
    # subtract ImageNet means before passing to ImageNet classifier
    X_trainR = X_train[i]*255.0-123.68
    X_trainG = X_train[i]*255.0-116.78
    X_trainB = X_train[i]*255.0-103.94
    X_train_feat[i] = vgg.predict(np.stack((X_trainR,X_trainG,X_trainB), axis=-1).reshape((1,224,224,3)))
    if i % 5000 == 0:
        print "i =", i
X_train_feat = np.reshape(X_train_feat, (X_train.shape[0], 7*7*512))

In [ ]:
# fully connected
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7*7*512))
model.add(layers.Dropout(0.9))
model.add(layers.Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(X_train_feat, y_train,
          validation_split=0.2,
          batch_size=100,
          epochs=EPOCHS,
          verbose=1,
          shuffle=True
          )

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Single-Frame VGG-Pretrained Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# calculate features for test data
X_test_feat = np.zeros(shape=(X_test.shape[0],7,7,512))
for i in range(X_test.shape[0]):
    X_testR = X_test[i]*255.0-123.68
    X_testG = X_test[i]*255.0-116.78
    X_testB = X_test[i]*255.0-103.94
    X_test_feat[i] = vgg.predict(np.stack((X_testR,X_testG,X_testB), axis=-1).reshape(1,224,224,3))
X_test_feat = np.reshape(X_test_feat, (X_test.shape[0], 7*7*512))

test_loss, test_acc = model.evaluate(X_test_feat, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
# take average prediction, round at end
frame_pred = np.reshape(model.predict(X_test_feat), (len(test_files)//FRAMES_PER_VID, FRAMES_PER_VID))
simple_y_pred = np.round(np.sum(frame_pred, axis=1)/FRAMES_PER_VID)
avg_acc = metrics.accuracy_score(y_test[::FRAMES_PER_VID], simple_y_pred)

print('Per-Video AVG Accuracy:', avg_acc)

# Multi-Frame Classification

## Loading Data

In [ ]:
# perform leave-one-out classification for each user in turn
#for user in users:
for user in ["alex"]:
    print "User:", user
    
    # training data consists of swipes from all other users (for mats of interest)
    print "\tFinding training videos..."
    files = []
    training_users = [u for u in users if u != user]
    for train_user in training_users:
        for mat in mats:
            for pres in pressures:
                folder = "data/"+train_user+"/"+frames+"/"+mat+"/"+pres+"/"
                files.extend([folder+f for f in os.listdir(folder)])
    files.sort() # only keeps video frames in order if < 10 frames per video
    print "\tTraining videos found:", len(files)//FRAMES_PER_VID
    
    # create single arrays to store data (use 224x224 for VGGNet)
    X_train = np.ndarray(shape=(len(files)*AUG_FACTOR/FRAMES_PER_VID, FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)
    y_train = np.ndarray(shape=(len(files)*AUG_FACTOR/FRAMES_PER_VID, 1), dtype=np.float32)

    # load in training data from images into normalized array
    print "\tLoading and augmenting training videos..."
    n = 0
    for i, filename in enumerate(files):
        # only act once for each video (filenames were sorted earlier)
        if i % FRAMES_PER_VID != 0:
            continue
        
        # extract section of filename which determines swipe
        match = re.match("([a-z/_]+[0-9]+_)([0-9]+)(.jpg)", filename, re.I)
        if match:
            swipe = match.groups()[0]
            frame = int(match.groups()[1])
            ext = match.groups()[2]
          
            # read in all other video frames from that swipe
            for f in range(FRAMES_PER_VID):
                x = cv2.resize(cv2.imread(swipe+str(f+1)+ext, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE,IMAGE_SIZE))
                X_train[i//FRAMES_PER_VID,f,:,:,0] = x / 255.0

                # perform data augmentation
                X_train[n+1,f,:,:,0] = np.fliplr(X_train[n,f,:,:,0])
                X_train[n+2,f,:,:,0] = np.rot90(X_train[n,f,:,:,0])
                X_train[n+3,f,:,:,0] = np.fliplr(X_train[n+2,f,:,:,0])
                X_train[n+4,f,:,:,0] = np.rot90(X_train[n+2,f,:,:,0])
                X_train[n+5,f,:,:,0] = np.fliplr(X_train[n+4,f,:,:,0])
                X_train[n+6,f,:,:,0] = np.rot90(X_train[n+4,f,:,:,0])
                X_train[n+7,f,:,:,0] = np.fliplr(X_train[n+6,f,:,:,0])

            # assign labels
            if "/hard/" in swipe:
                y_train[n:n+AUG_FACTOR] = np.ones((AUG_FACTOR,1))
            else:
                y_train[n:n+AUG_FACTOR] = np.zeros((AUG_FACTOR,1))
            n += AUG_FACTOR
    print "\tAll", n//FRAMES_PER_VID, "training videos and labels stored successfully!"
    
    # find all test data
    print "\tFinding test videos..."
    test_files = []
    for mat in mats:
        for pres in pressures:
            folder = "data/"+user+"/"+frames+"/"+mat+"/"+pres+"/"
            test_files.extend([folder+f for f in os.listdir(folder)])
    test_files.sort() # only keeps video frames in order if < 10 frames per video
    print "\tTest videos found:", len(test_files)//FRAMES_PER_VID
    
    # create arrays to store test data
    X_test = np.ndarray(shape=(len(test_files)*AUG_FACTOR/FRAMES_PER_VID, FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)
    y_test = np.ndarray(shape=(len(test_files)*AUG_FACTOR/FRAMES_PER_VID, 1), dtype=np.float32)

    print "\tLoading test videos..."
    for i, filename in enumerate(test_files):
        # only act once for each video (filenames were sorted earlier)
        if i % FRAMES_PER_VID != 0:
            continue
        
        # extract section of filename which determines swipe
        match = re.match("([a-z/_]+[0-9]+_)([0-9]+)(.jpg)", filename, re.I)
        if match:
            swipe = match.groups()[0]
            frame = int(match.groups()[1])
            ext = match.groups()[2]
          
            # read in all other video frames from that swipe
            for f in range(FRAMES_PER_VID):
                x = cv2.resize(cv2.imread(swipe+str(f+1)+ext, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE,IMAGE_SIZE))
                X_test[i//FRAMES_PER_VID,f,:,:,0] = x / 255.0
            y_test[i//FRAMES_PER_VID] = float("/hard/" in swipe)
    print "\tAll", i//FRAMES_PER_VID, "test images and labels stored successfully!"

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = [fig.add_subplot(2,4,j+1) for j in range(8)]
for j in range(8):
    plt.subplot(2, 4, j+1)
    plt.imshow(X_train[2,j,:,:,0], cmap='gray', vmin=0, vmax=1)
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')
fig.subplots_adjust(wspace=0, hspace=0)

## Calculate Optical Flow

Generate visual of optical flow over time

In [ ]:
S = 2
fig = plt.figure(figsize=(8,4))
ax = [fig.add_subplot(2,4,j+1) for j in range(8)]
plt.subplot(2,4,1)
plt.imshow(np.ones((IMAGE_SIZE,IMAGE_SIZE,3))*-1)
for j in range(1,8):
    plt.subplot(2, 4, j+1)
    hsv = np.uint8(np.zeros((X_train.shape[2], X_train.shape[3], 3)))
    hsv[...,1] = 255
    prv = X_train[S,j-1,:,:,0]
    nxt = X_train[S,j,:,:,0]
    flow = cv2.calcOpticalFlowFarneback(prv, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = cv2.normalize(ang,None,0,255,cv2.NORM_MINMAX)
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    plt.imshow(bgr)
    
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')
fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
X_train_flowx = np.ndarray(shape=(X_train.shape[0], FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)
X_train_flowy = np.ndarray(shape=(X_train.shape[0], FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)

# calculate optical flow for each training example
print "Calculating optical flow for training videos..."
for i in range(X_train.shape[0]):
    for j in range(1,FRAMES_PER_VID):
        prv = X_train[i,j-1,:,:,0]
        nxt = X_train[i,j,:,:,0]
        flow = cv2.calcOpticalFlowFarneback(prv, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        X_train_flowx[i,j,:,:,0] = flow[:,:,0]
        X_train_flowy[i,j,:,:,0] = flow[:,:,1]
        
# to keep dims same, set flow result for first frame same as second
X_train_flowx[:,0,:,:,0] = X_train_flowx[:,1,:,:,0]
X_train_flowy[:,0,:,:,0] = X_train_flowy[:,1,:,:,0]
print "Optical flow calculated for all training videos!"


X_test_flowx = np.ndarray(shape=(X_test.shape[0], FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)
X_test_flowy = np.ndarray(shape=(X_test.shape[0], FRAMES_PER_VID, IMAGE_SIZE, IMAGE_SIZE, 1), dtype=np.float32)

# calculate optical flow for each test example
print "Calculating optical flow for test videos..."
for i in range(X_test.shape[0]):
    for j in range(1,FRAMES_PER_VID):
        prv = X_test[i,j-1,:,:,0]
        nxt = X_test[i,j,:,:,0]
        flow = cv2.calcOpticalFlowFarneback(prv, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        X_test_flowx[i,j,:,:,0] = flow[:,:,0]
        X_test_flowy[i,j,:,:,0] = flow[:,:,1]
        
# to keep dims same, set flow result for first frame same as second
X_test_flowx[:,0,:,:,0] = X_test_flowx[:,1,:,:,0]
X_test_flowy[:,0,:,:,0] = X_test_flowy[:,1,:,:,0]
print "Optical flow calculated for all test videos!"

## Calculate Fade

In [ ]:
S = 1
fig = plt.figure(figsize=(8,4))
ax = [fig.add_subplot(2,4,j+1) for j in range(8)]
plt.subplot(2,4,1)
plt.imshow(np.ones((IMAGE_SIZE,IMAGE_SIZE,3))*-1)
for j in range(1,8):
    plt.subplot(2, 4, j+1)
    prv = X_train[S,j-1,:,:,0]
    nxt = X_train[S,j,:,:,0]
    diff = prv - nxt
    diff[diff<=0.01] = -1 # ignore regions that increased in intensity
    diff[diff>0.5] = -1 # ignore finger -> no finger pixel transition
    plt.imshow(diff, cmap='gray', vmin=-1, vmax=1)
    
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')
fig.subplots_adjust(wspace=0, hspace=0)

## Multi-Frame 3D Convolution

Model borrowed from C3D paper, https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2

In [ ]:
model = models.Sequential()
# 0th layer group
model.add(layers.Conv3D(32, (3,3,3), activation='relu', padding='same', 
                        input_shape=(8,IMAGE_SIZE,IMAGE_SIZE,1)))
model.add(layers.MaxPooling3D(pool_size=(2,2,2), strides=(1,2,2), ))
# 1st layer group
model.add(layers.Conv3D(32, (3,3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
# 2nd layer group
model.add(layers.Conv3D(32, (3,3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
# 3rd layer group
#model.add(layers.Conv3D(32, (3,3,3), activation='relu', padding='same'))
#model.add(layers.Conv3D(32, (3,3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2)))
# 4th layer group
#model.add(layers.Conv3D(64, (3,3,3), activation='relu', padding='same'))
#model.add(layers.Conv3D(64, (3,3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
# 5th layer group
#model.add(layers.Conv3D(64, (3,3,3), activation='relu', padding='same'))
#model.add(layers.Conv3D(64, (3,3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2)))

# fully connected
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
          validation_split=0.2,
          batch_size=100,
          epochs=EPOCHS,
          verbose=1
          )

## Testing Subsampling

In [ ]:
S = 2
STEP = 4
fig = plt.figure(figsize=(8,4))
ax = [fig.add_subplot(2,4,j+1) for j in range(8)]
ss = np.zeros((8,IMAGE_SIZE//STEP,IMAGE_SIZE//STEP))
for f in range(8):
    for i in range(0,IMAGE_SIZE,STEP):
        for j in range(0,IMAGE_SIZE,STEP):
            ss[f,i//STEP,j//STEP] = np.sum(X_train[S,f,i:i+STEP,j:j+STEP])
            
for j in range(8):
    plt.subplot(2, 4, j+1)
    plt.imshow(ss[j], cmap='gray', vmin=-STEP*STEP, vmax=STEP*STEP)
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')
fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
256/4